### Package Installation

Package installation:

In [11]:
%%capture
#!pip uninstall gspread_models
!pip install gspread_models==1.0.2 # --upgrade

In [12]:
!pip list | grep gspread_models

gspread_models                   1.0.2


### Google Credentials

In [13]:
from google.colab import auth

auth.authenticate_user()

In [14]:
from google.auth import default

creds, _ = default()
type(creds)

google.auth.compute_engine.credentials.Credentials

### Spreadsheet Service

Create a Google Sheets document with "books" sheet, with columns: `id`, `title`, `author`, `year`, and `created_at`. Note the document identifier, and set it as the `GSPREAD_MODELS_DOCUMENT_ID` notebook secret.

In [15]:
from google.colab import userdata

GOOGLE_SHEETS_DOCUMENT_ID = userdata.get("GSPREAD_MODELS_DOCUMENT_ID")

Build a new service instance:

In [16]:
from gspread_models.service import SpreadsheetService

service = SpreadsheetService(creds=creds, document_id=GOOGLE_SHEETS_DOCUMENT_ID)

SPREADSHEET SERVICE...
DOCUMENT ID: 1Nk-UBd-3TyWZqbWSvKIIF-S_KKdrkVFRVeNrFy_F1gs


In [17]:
service.doc

<Spreadsheet 'GSpread Models (Test Database)' id:1Nk-UBd-3TyWZqbWSvKIIF-S_KKdrkVFRVeNrFy_F1gs>

In [18]:
for sheet in service.sheets:
    print(sheet.title)

Info
products
orders
books


### Binding the Model

Bind the base model with access to your account credentials and your document:

In [21]:
from gspread_models.base import BaseModel

BaseModel.service = service
#BaseModel.set_document_id(GOOGLE_SHEETS_DOCUMENT_ID) # not necessary since we are binding at the service level

### Model Class

Now, defining custom class that inherits from the base model:

In [22]:
class Book(BaseModel):

    SHEET_NAME = "books"

    COLUMNS = ["title", "author", "year"]

    SEEDS = [
        {"title": "To Kill a Mockingbird", "author": "Harper Lee", "year": 1960},
        {"title": "1984", "author": "George Orwell", "year": 1949},
        {"title": "The Great Gatsby", "author": "F. Scott Fitzgerald", "year": 1925},
        {"title": "The Catcher in the Rye", "author": "J.D. Salinger", "year": 1951},
        {"title": "Pride and Prejudice", "author": "Jane Austen", "year": 1813},
        {"title": "To the Lighthouse", "author": "Virginia Woolf", "year": 1927},
        {"title": "The Hobbit", "author": "J.R.R. Tolkien", "year": 1937},
        {"title": "Moby-Dick", "author": "Herman Melville", "year": 1851},
        {"title": "Brave New World", "author": "Aldous Huxley", "year": 1932},
        {"title": "Alice's Adventures in Wonderland", "author": "Lewis Carroll", "year": 1865},
        {"title": "Harry Potter and the Philosopher's Stone", "author": "J.K. Rowling", "year": 1997},
        {"title": "Harry Potter and the Chamber of Secrets", "author": "J.K. Rowling", "year": 1998},
    ]

### Queries

Destroy all records:

In [23]:
Book.destroy_all()

SHEET ('books')...


{'spreadsheetId': '1Nk-UBd-3TyWZqbWSvKIIF-S_KKdrkVFRVeNrFy_F1gs',
 'replies': [{}]}

Populate sheet with default records:

In [24]:
Book.seed()

SHEET ('books')...


{'spreadsheetId': '1Nk-UBd-3TyWZqbWSvKIIF-S_KKdrkVFRVeNrFy_F1gs',
 'tableRange': 'books!A1:F1',
 'updates': {'spreadsheetId': '1Nk-UBd-3TyWZqbWSvKIIF-S_KKdrkVFRVeNrFy_F1gs',
  'updatedRange': 'books!A2:E13',
  'updatedRows': 12,
  'updatedColumns': 5,
  'updatedCells': 60}}

Fetch all records from the sheet:

In [25]:
books = Book.all()
len(books)

SHEET ('books')...


12

In [30]:
for book in books:
    print(book.id, book.title, book.author, sep=" | ")
    # book.created_at.strftime("%Y-%m-%d")

1 | To Kill a Mockingbird | Harper Lee
2 | 1984 | George Orwell
3 | The Great Gatsby | F. Scott Fitzgerald
4 | The Catcher in the Rye | J.D. Salinger
5 | Pride and Prejudice | Jane Austen
6 | To the Lighthouse | Virginia Woolf
7 | The Hobbit | J.R.R. Tolkien
8 | Moby-Dick | Herman Melville
9 | Brave New World | Aldous Huxley
10 | Alice's Adventures in Wonderland | Lewis Carroll
11 | Harry Potter and the Philosopher's Stone | J.K. Rowling
12 | Harry Potter and the Chamber of Secrets | J.K. Rowling


Find record by id:

In [32]:
book = Book.find(3)
dict(book)

SHEET ('books')...


{'id': 3,
 'title': 'The Great Gatsby',
 'author': 'F. Scott Fitzgerald',
 'year': 1925,
 'created_at': datetime.datetime(2024, 3, 27, 1, 7, 50, 916904, tzinfo=datetime.timezone.utc)}

Filter records on matching conditions:


In [33]:
book = Book.where(author="F. Scott Fitzgerald")[0]
dict(book)

SHEET ('books')...


{'id': 3,
 'title': 'The Great Gatsby',
 'author': 'F. Scott Fitzgerald',
 'year': 1925,
 'created_at': datetime.datetime(2024, 3, 27, 1, 7, 50, 916904, tzinfo=datetime.timezone.utc)}

In [34]:
hp_books = Book.where(author="J.K. Rowling")
for book in hp_books:
    print(book.id, book.title, book.author, sep=" | ")

SHEET ('books')...
11 | Harry Potter and the Philosopher's Stone | J.K. Rowling
12 | Harry Potter and the Chamber of Secrets | J.K. Rowling
